In [32]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
from copy import deepcopy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
# from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
# from torchtext.vocab import build_vocab_from_iterator
# import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
import warnings

In [24]:
class PositionalEncoding(nn.Module):
    '''
    Implement the PE function.
    d_model: 输入序列中每个词嵌入的维度（与词向量的维度一致）
    '''
    
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)
        
        # compute the positional encoding once in log space.
        # 初始化一个大小为 [max_len, d_model] 的全零张量 pe，用于存储位置编码的值。
        pe = torch.zeros(max_len, d_model)
        # torch.arange 是 PyTorch 中用于生成等间隔数值序列的一种函数。其功能类似于 Python 中的 range 函数，但它生成的是张量（tensor）而不是普通的数字序列。
        # 生成一个 position 张量，它的值是从 0 到 max_len-1 的序列（即每个位置的索引），形状为 [max_len, 1]。unsqueeze(1) 表示在第一维度增加一个维度。
        position = torch.arange(0, max_len).unsqueeze(1) # [max_len, 1]
        '''
        计算位置编码中的分母项 div_term, 使用上述公式, 保存为 div_term = 1/分母。
        torch.arange(0, d_model, 2) 生成从 0 开始到 d_model-1 步长为 2 的序列，这部分用来处理偶数位置的维度。
        
        为什么是公式: torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model) ?

        '''
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term) # (0, 2, 4, ...)
        pe[:, 1::2] = torch.cos(position * div_term) # (1, 3, 5, ...)
        
        # 扩展维度并注册为 buffer
        # 使用 unsqueeze(0) 为位置编码增加一个批次维度, 这样可以适配不同批次的数据。
        pe = pe.unsqueeze(0) # [max_len, d_model] -> [1, max_len, d_model]
        
        '''
        将 pe 作为 buffer 注册到模型中。Buffer 是模型的持久状态，但不会作为模型参数参与优化，也不会随着梯度更新而改变。
        通常情况下，模型中的状态分为两类：
            可训练参数：这些参数会在模型的前向传播和反向传播过程中参与梯度计算，并在优化器中更新。这些参数可以通过 nn.Parameter 来注册，它们通常是权重或偏置。通过优化器或手动调整更新
            非可训练的持久状态：这些是模型需要的值，但它们不需要参与优化过程。只能通过代码逻辑手动修改。例如： 
                用于存储固定的常量，比如用于正则化的参数、批次统计数据（如 BatchNorm 中的均值和方差）。
                用于模型中不需要梯度更新但要随模型一起保存的变量。
        Buffer 就属于第二类状态。它的关键点是：
            不会参与梯度计算，不会随着训练过程中的优化步骤被更新。
            会随模型保存和加载，即使这些变量不会更新，仍然希望它们作为模型的一部分存储和恢复。
        持久存储: pe 是一个位置编码矩阵，代表了每个位置的编码。这个矩阵是预先计算的，并且在整个模型中保持不变，因此它不需要被优化器更新。
                但在保存模型时，我们希望位置编码 pe 和其他可训练的参数一样，被保存下来，并在加载模型时恢复。
        不会更新：由于位置编码的矩阵 pe 是基于固定的公式计算的，它不需要参与反向传播和梯度更新。
                因此，将其作为 buffer 来注册，这样在训练过程中它不会被优化器错误地修改。
        '''
         
        self.register_buffer("pe", pe)
        
    def forward(self, x):
        # x: [batch_size, sequence_length, d_model]
        # 这一步是将位置编码加到词嵌入上，使得模型能够感知位置信息。
        # requires_grad_(False) 是为了防止位置编码参与梯度更新，因为它们是固定的。
        x =  x + self.pe[:, x.size(1)].requires_grad_(False)
        
        '''
        为什么对位置编码应用 Dropout ? 
            尽管 Dropout 最初是在全连接层中用于“随机失活”一部分神经元的，但它的应用范围实际上更广，既可以用于神经网络的各层、或输出层，也可以用于特征向量，如输入的词嵌入或位置编码。
            增加随机性: 通过对位置编码的部分值进行随机“失活”，可以引入一些随机性，使得模型不会过度依赖特定的位置信息。这可以让模型在训练时更具鲁棒性，减少过拟合的可能。
            正则化输入: Dropout 可以防止模型对特定的输入位置编码产生过度拟合。因为位置编码是添加到输入中的一部分，
                      对其进行 dropout 可以有效地打破模型对绝对位置信息的依赖性，迫使模型更关注整体上下文，而不仅仅是某些特定位置的依赖。
        '''
        return self.dropout(x)

### Positional Encoding

<img src="./images/Position.png" alt="示例图片" width="700">

<img src="./images/Position2.png" alt="示例图片" width="300">

$\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad PE_{(pos, 2i)} = \sin(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$

$\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad PE_{(pos, 2i+1)} = \cos(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$

$\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad$ 其中 2i 和 2i+1 是特征的维度第几维，分为第**奇**数维和第**偶**数维

<img src="./images/Position3.png" alt="示例图片" width="700">

# 绝对位置编码

In [25]:
class SinPositionEncoding(nn.Module):
    def __init__(self, max_sequence_length, d_model, base=10000):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model
        self.base = base
        
    def forward(self):
        # 初始化一个大小为 [max_len, d_model] 的全零张量 pe，用于存储位置编码的值。
        pe = torch.zeros(self.max_sequence_length, self.d_model, dtype=torch.float)
        
        exp_1 = torch.arange(self.d_model//2, dtype=torch.float) # 初始化一半维度，sin位置编码的维度被分成了两部分
        
        exp_value = 2 * exp_1 / self.d_model
        
        alpha = 1 / (self.base ** exp_value) # size(d_model / 2)
        
        out = torch.arange(self.max_sequence_length, dtype=torch.float)[:, None] @ alpha[None, :] 
        # size(max_sequence_length, d_model / 2)
        # [:, None] 给张量新添加了一个维度 (max_sequence_length, 1)
        # [None, :] (1, d_model / 2)
        # @: 矩阵乘法
        
        embedding_sin = torch.sin(out)
        embedding_cos = torch.cos(out)
        
        pe[:, 0::2] = embedding_sin # 将 embedding_sin 的内容赋值到 pe 的偶数列上
        # 行数 (max_sequence_length)：embedding_sin 的行数与 pe 相同，每一行对应一个位置。
        # embedding_sin 的列数是 pe 的一半，因为正弦部分只填充到 pe 的偶数列 (0::2)。
        pe[:, 1::2] = embedding_cos
        
        return pe
    
SinPositionEncoding(d_model=4, max_sequence_length=10, base=10000).forward()
        

tensor([[ 0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0100,  1.0000],
        [ 0.9093, -0.4161,  0.0200,  0.9998],
        [ 0.1411, -0.9900,  0.0300,  0.9996],
        [-0.7568, -0.6536,  0.0400,  0.9992],
        [-0.9589,  0.2837,  0.0500,  0.9988],
        [-0.2794,  0.9602,  0.0600,  0.9982],
        [ 0.6570,  0.7539,  0.0699,  0.9976],
        [ 0.9894, -0.1455,  0.0799,  0.9968],
        [ 0.4121, -0.9111,  0.0899,  0.9960]])

# 可学习位置编码

In [30]:
class TrainablePositionEncoding(nn.Module):
    def __init__(self, max_sequence_length, d_model):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model
        
    def forward(self):
        pe = nn.Embedding(self.max_sequence_length, self.d_model)
        # pe 是一个 torch.nn.Embedding 或类似的层，它的 weight 属性是一个可以学习的权重矩阵。
        nn.init.constant(pe.weight, 0.) # 将 pe.weight 的所有元素初始化为 0。
        
        return pe
pe = TrainablePositionEncoding(max_sequence_length=10, d_model=4).forward()

/var/folders/mn/8fw7vgd551394dxtykrf78200000gn/T/ipykernel_36378/3846723043.py:10: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(pe.weight, 0.) # 将 pe.weight 的所有元素初始化为 0。


# 相对位置编码

<img src="./images/xiangdui.png" alt="示例图片" width="1100">

In [34]:
class RelativePosition(nn.Module):
    '''
    根据查询和键的长度，生成相对位置的差距矩阵。
    将差距裁剪到指定范围并映射为合法索引。
    根据索引从嵌入表中查找相应的嵌入，输出相对位置编码。
    '''
    def __init__(self, num_units, max_relative_position, device = 'cpu'):
        super().__init__()
        self.num_units = num_units # 每个相对位置的嵌入维度
        self.max_relative_position = max_relative_position # 最大相对距离
        self.embeddings_table = nn.Parameter(torch.Tensor(max_relative_position * 2 + 1, num_units))
        # self.embeddings_table 是一个 learnable 参数矩阵, 形状为 (max_relative_position * 2 + 1, num_units)
        # 如果 self.embeddings_table 的形状是 (5, num_units)，
        # 而 final_mat 是一个形状为 (length_q, length_k) 的张量，其值是 [0, 1, 2, 3, 4] 之间的索引。
        # 那么 self.embeddings_table[final_mat] 会返回形状为 (length_q, length_k, num_units) 的张量，对应每个相对位置的嵌入。

        nn.init.xavier_uniform_(self.embeddings_table)
        self.device = device
        
    def forward(self, length_q, length_k):
        range_vec_q = torch.arange(length_q)
        range_vec_k = torch.arange(length_k)
        # 一般 length_q == length_k
        # 通过广播机制,计算相对位置差矩阵, (length_q, length_k)
        distance_mat = range_vec_q[None, :] - range_vec_k[:, None]
        
        # 将超出范围的差距裁剪到 [-max_relative_position, max_relative_position]
        distance_mat_clipped = torch.clamp(distance_mat, -self.max_relative_position, self.max_relative_position)
        
        # 将 [-2, -1, 0, 1, 2] 映射为 [0, 1, 2, 3, 4]
        # final_mat 是一个整型索引张量，用于从 self.embeddings_table 中查找对应的嵌入
        final_mat = distance_mat_clipped + self.max_relative_position
        final_mat = torch.LongTensor(final_mat).to(self.device)
        
        embeddings = self.embeddings_table[final_mat].to(self.device)
        
        return embeddings

### Multi Head Attention

<img src="./images/MultiHeadAttention.png" alt="示例图片" width="700">

$MultiHead(Q, K, V) = Concat(head_1, head_2, ... , head_h)W^O\text{, where }head_i = Attention(QW_i^Q, KW_i^K, VW_i^V)$

$W_i^Q \in \mathbb{R}^{d_{model} \times d_k}, \quad W_i^K \in \mathbb{R}^{d_{model} \times d_K}, \quad W_i^V \in \mathbb{R}^{d_{model} \times d_V}, \quad W_i^O \in \mathbb{R}^{hd_v \times d_{model}}$

$$
Attention(q, k, v) = [softmax(\frac{q \dot (k + pos_k)}{\sqrt{d_k}})](v + pos_v)
$$

其中 $pos_k$ 是注入键（$k$）的相对位置编码。只需要为 $k$ 注入相对位置信息，因为 𝑞⋅𝑘 已能体现注意力权重的相对顺序。

相对位置编码的核心目标是调整键（k）和值（v）的权重。

注入到 k 中的相对位置编码即可覆盖所有信息，q 不需要重复注入。

In [40]:
def _get_clones(module, num):
        """返回指定数量的 module 深拷贝"""
        return nn.ModuleList([deepcopy(module) for _ in range(num)])

class RelativeMultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1, batch_size=6, device='cpu'):
        "Take in model size and number of heads. "
        super(RelativeMultiHeadAttention, self).__init__()
        self.device = device
        self.d_model = d_model
        self.n_heads = n_heads
        self.batch_size = batch_size
        
        assert d_model % n_heads == 0
        self.head_dim = d_model // n_heads
        
        self.linears = _get_clones(nn.Linear(d_model, d_model), 4)
        # linears: W_i^Q, W_i^K, W_i^V, W_i^O
        self.dropout = nn.Dropout(p=dropout)
        self.relative_position_k = RelativePosition(self.head_dim, max_relative_position=16)
        self.relative_position_v = RelativePosition(self.head_dim, max_relative_position=16)
        # 返回形状为 (length_q, length_k, head_dim) 的张量，对应每个相对位置的嵌入
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value):
        # embedding
        # query, key, value = [batch_size, len, hid_dim]
        # 1) Do all the linear projections in batch from d_model => h x d_k
        '''
        这段代码的作用是通过线性变换，将 query、key 和 value 映射到多头注意力机制的特征空间。
        .view(self.batch_size, -1, self.d_model):
            .view(self.batch_size, -1, self.d_model)：将线性变换的结果重塑为 (self.batch_size, -1, self.d_model) 的形状，
            目的是分配给每个头。
            view 函数用于改变张量的形状。这里的目的是将每个线性层的输出重塑为形状为 [self.batch_size, -1, self.d_model] 的张量。
            batch_size 是批次大小，表示每个批次中的样本数。
            -1 是自动推导的维度，它会根据原始张量的总元素数量和指定的其他维度自动计算,即 len。
            self.d_model 是特征维度。
        zip 函数将 self.linears 和 (query, key, value) 这三个张量打包在一起，使得在循环中可以同时迭代线性层和输入张量。
            三个线性层, 分别以query, key, value作为参数跑一遍
        '''
        query, key, value = [l(x).view(self.batch_size, -1, self.d_model) for l,x in zip(self.linears, (query, key, value))]
        
        len_k = query.shape[1]
        len_q = query.shape[1]
        len_v = value.shape[1]
        
        # Self-Attention
        # r_q1, r_k1 = [batch_size, len, n_heads, head_dim]
        # -> [batch_size, n_heads, len, head_dim]
        r_q1 = query.view(self.batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        r_k1 = key.view(self.batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        # -> [batch_size, n_heads, head_dim, len]
        # 计算 q*k
        attn1 = torch.matmul(r_q1, r_k1.permute(0, 1, 3, 2))
        # attn1: [batch_size, n_heads, head_dim, len(_q), len(_k)]
        
        # [batch_size, len, d_model] -> [len, batch_size, d_model] (-> [len, batch_size, n_heads, head_dim])
        # -> [len, batch_size * n_heads, head_dim]
        # batch_size * n_heads 合并批量大小和头的数量，适配批量化并行处理
        r_q2 = query.permute(1, 0, 2).contiguous().view(len_q, self.batch_size * self.n_heads, self.head_dim)
        # contiguous的作用是 保证张量在内存中是连续存储的。
        # 在某些张量操作（比如 permute 或 view）之后，张量可能不再是连续的，
        # 这时需要调用 contiguous() 来创建一个连续的张量副本。
        # permute 作用: 改变张量的维度顺序以适配逻辑需求。
        # view 作用: 根据新的需求调整张量的形状。
        # 为什么要先 permute 再 view？
        # 因为 view 要求张量是连续的，而 permute 不改变内存布局，因此必须先 permute，再用 contiguous() 确保内存连续，最后用 view 调整形状。
        r_k2 = self.relative_position_k(len_q, len_k)
        # r_k2: [len_q, len_k, head_dim]
        
        # 计算 q*pos_k
        # [len, batch_size * n_heads, head_dim], [len_q, len_k, head_dim]->[len_q, head_dim, len_k]
        # attn2: [len_q, batch_size * n_heads, len_k] -> [batch_size * n_heads, len_q, len_k]
        attn2 = torch.matmul(r_q2, r_k2.transpose(1, 2)).transpose(0, 1)
        attn2 = attn2.contiguous().view(self.batch_size, self.n_heads, len_q, len_k)
        # attn2: [batch_size, n_heads, len_q, len_k]
        
        attn = (attn1 + attn2) / self.scale
        attn = self.dropout(torch.softmax(attn, dim=-1))
        # attn: [batch_size, n_heads, len(_q), len(_k)]
        
        # 同上
        r_v1 = value.view(self.batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        # [batch_size, n_heads, head_dim, len]
        
        # 计算 attn*v
        # [batch_size, n_heads, len(_q), len(_k)], [batch_size, n_heads, head_dim, len]
        weight1 = torch.matmul(attn, r_v1)
        # weight1: [batch_size, n_heads, len, len]
        
        r_v2 = self.relative_position_v(len_q, len_v)
        # r_v2: [len_q, len_v, head_dim]
        
        # [batch_size, n_heads, len(_q), len(_k)] -> [len(_q), batch_size, n_heads, len(_k)]
        # weight2 -> [len(_q), batch_size * n_heads, len(_k)]
        weight2 = attn.permute(2, 0, 1, 3).contiguous().view(len_q, self.batch_size * self.n_heads, len_k) # len_k == len_v
        # [len(_q), batch_size * n_heads, len(_k)], [len_q, len_v, head_dim]
        # weight2 -> [len(_q), batch_size * n_heads, head_dim]
        weight2 = torch.matmul(weight2, r_v2)
        
        # [len(_q), batch_size * n_heads, head_dim] (-> [batch_size * n_heads, len(_1), head_dim])
        # weight2 -> [batch_size, n_heads, len(_q), head_dim]
        weight2 = weight2.transpose(0, 1).contiguous().view(self.batch_size, self.n_heads, len_q, self.head_dim)
        
        # x: [batch_size, n_heads, len(_q), head_dim]
        x = weight1 + weight2
        
        # x: [batch_size, len(_q), n_heads, head_dim]
        x = x.permute(0, 2, 1, 3).contiguous()
        
        # x: [batch_size * len(_q), n_heads, head_dim] -> [batch_size * len(_q), d_model]
        x = x.view(self.batch_size * len_q, self.d_model)
        
        return self.linears[-1](x)

In [41]:
if __name__ == "__main__":
    # Hyperparameters
    batch_size = 2
    seq_len = 5
    d_model = 16
    n_heads = 4
    device = 'cpu'

    # Dummy inputs
    query = torch.rand(batch_size, seq_len, d_model)
    key = torch.rand(batch_size, seq_len, d_model)
    value = torch.rand(batch_size, seq_len, d_model)

    # Initialize the model
    model = RelativeMultiHeadAttention(d_model=d_model, n_heads=n_heads, batch_size=batch_size, device=device)
    model.to(device)
    
    # Forward pass
    output = model(query, key, value)
    print("Output shape:", output.shape)

Output shape: torch.Size([10, 16])


# RoPE

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [43]:
def sinusoidal_position_embedding(batch_size, nums_head, max_len, output_dim, device):
    # (max_len, 1)
    position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(-1)
    # torch.arange 生成一个张量（tensor），该张量包含指定范围内的连续值
    # (output_dim // 2)
    ids = torch.arange(0, output_dim // 2, dtype=torch.float) # 即公式里的 i, i的范围是[0, d/2]
    theta = torch.pow(10000, -2 * ids / output_dim)
    # print(position.shape) # torch.Size([max_len, 1])
    # print(theta.shape) # torch.Size([output_dim // 2])
    
    # (max_len, output_dim//2)
    embeddings = position * theta # 即公式里的: pos / (10000^(2i/d))
    # print(embedding.shape) # torch.Size([max_len, output_dim//2])
    
    # torch.stack 将一组张量沿着新的维度进行连接的函数。它将多个张量堆叠在一起，返回一个新的张量，新的维度通常是增加了一个维度的张量。
    # (max_len, output_dim//2, 2)
    embeddings = torch.stack([torch.sin(embeddings), torch.cos(embeddings)], dim=-1)
    
    # torch.repeat(*sizes) sizes：一个整数元组，指定每个维度的重复次数。这个元组的长度应该与原始张量的维度数相同。
    # e.g. t.repeat(2, 3) 使得原始张量在第0维上重复了 2 次，在第1维上重复了 3 次
    # (bs, head, max_len, output_dim//2, 2)
    embeddings = embeddings.repeat(batch_size, nums_head, *([1] * len(embeddings.shape)))
    # 在第一维(bs)重复batch_size次，在第二维重复nums_head次，其他维度重复一次(不重复)
    # *([1] * len(embeddings.shape)) 创建一个列表，列表的长度等于 embeddings 的维度数(len(t.shape))，每个元素都是 1
    
    # (hs, head, max_len, output_dim)
    # reshape后就是偶数sin，奇数cos了（是(output_dim//2, 2)，第一列是奇，第二列是偶，所以按行拼接就是奇偶交叉）
    # 如果是(2, output_dim//2)则变成了所有sin在前，所有cos在后
    '''
    t = torch.tensor([[1, 3, 5], [2, 4, 6]])
    flattened_tensor = t.reshape(-1) # tensor([1, 3, 5, 2, 4, 6])
    flattened_tensor = t.T.reshape(-1) # tensor([1, 2, 3, 4, 5, 6])
    t = torch.tensor([[[1, 2], [3, 4],[5,6]]])
    flattened_tensor = t.reshape(-1) # tensor([1, 2, 3, 4, 5, 6])
    flattened_tensor = t.T.reshape(-1) # tensor([1, 3, 5, 2, 4, 6])
    '''
    embeddings = torch.reshape(embeddings, (batch_size, nums_head, max_len, output_dim))
    # print(embeddings.shape) # torch.Size([batch_size, nums_head, max_len, output_dim])
    
    embeddings = embeddings.to(device)
    return embeddings

<img src="./images/RoPE.png" alt="示例图片" width="1100">

$$
f_q(x_m, m) = 
\begin{pmatrix}
\cos m\theta & -\sin m\theta \\
\sin m\theta & \cos m\theta
\end{pmatrix}
\begin{pmatrix}
W_q^{(1,1)} & W_q^{(1,2)} \\
W_q^{(2,1)} & W_q^{(2,2)}
\end{pmatrix}
\begin{pmatrix}
x_m^{(1)} \\
x_m^{(2)}
\end{pmatrix}
=
\begin{pmatrix}
\cos m\theta & -\sin m\theta \\
\sin m\theta & \cos m\theta
\end{pmatrix}
\begin{pmatrix}
q_m^{(1)} \\
q_m^{(2)}
\end{pmatrix}
$$

For n-dim:
$$
f_q(x_m, m) = 
\begin{pmatrix}
\cos m\theta_0 & -\sin m\theta_0 & 0 & 0 & \cdots & 0 & 0 \\
\sin m\theta_0 & \cos m\theta_0 & 0 & 0 & \cdots & 0 & 0 \\
0 & 0 & \cos m\theta_1 & -\sin m\theta_1 & \cdots & 0 & 0 \\
0 & 0 & \sin m\theta_1 & \cos m\theta_1 & \cdots & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \cdots & \cos m\theta_{d/2-1} & -\sin m\theta_{d/2-1} \\
0 & 0 & 0 & 0 & \cdots & \sin m\theta_{d/2-1} & \cos m\theta_{d/2-1}
\end{pmatrix}
\begin{pmatrix}
q_m^{(1)} \\
q_m^{(2)} \\
q_m^{(3)} \\
q_m^{(4)} \\
\vdots
\end{pmatrix}
$$

i.e.

$$
R_{\Theta, m}^d \mathbf{x} =
\begin{pmatrix}
x_0 \\
x_1 \\
x_2 \\
x_3 \\
\vdots \\
x_{d-2} \\
x_{d-1}
\end{pmatrix}
\otimes
\begin{pmatrix}
\cos m\theta_0 \\
\cos m\theta_0 \\
\cos m\theta_1 \\
\cos m\theta_1 \\
\vdots \\
\cos m\theta_{d/2-1} \\
\cos m\theta_{d/2-1}
\end{pmatrix}
+
\begin{pmatrix}
-x_1 \\
x_0 \\
-x_3 \\
x_2 \\
\vdots \\
-x_{d-1} \\
x_{d-2}
\end{pmatrix}
\otimes
\begin{pmatrix}
\sin m\theta_0 \\
\sin m\theta_0 \\
\sin m\theta_1 \\
\sin m\theta_1 \\
\vdots \\
\sin m\theta_{d/2-1} \\
\sin m\theta_{d/2-1}
\end{pmatrix}
$$

In [44]:
def RoPE(q, k):
    assert(q.shape == k.shape)
    
    # q, k: (bs, head, max_len, output_dim)
    batch_size = q.shape[0]
    nums_head = q.shape[1]
    max_len = q.shape[2]
    output_dim = q.shape[-1]
    
    # (bs, head, max_len, output_dim)
    pos_emb = sinusoidal_position_embedding(batch_size, nums_head, max_len, output_dim, q.device)
    
    # cos_pos, sin_pos: (bs, head, max_len, output_dim)
    # 看rope公式可知，相邻一组(如1-2，3-4，5-6，...)之间的 \theta 是相同的，所以cos(m\theta)，sin(m\theta)也是同一个。
    # 只需要复制一遍sin，cos的向量即可,如(1, 2, 3)变成(1, 1, 2, 2, 3, 3)
    # interleave 交错
    # torch.repeat_interleave(input, repeats, dim=None) 每个元素重复 repeats 次
    # pos_emb中先sin后cos交错排列
    sin_pos = pos_emb[..., 0::2].repeat_interleave(2, dim=-1)
    cos_pos = pos_emb[..., 1::2].repeat_interleave(2, dim=-1)
    
    # q,k: (bs, head, max_len, output_dim)
    q2 = torch.stack([-q[..., 1::2], q[..., 0::2]], dim=-1)
    # q2: (bs, head, max_len, output_dim//2, 2)
    q2 = q2.reshape(q.shape) # reshape 后就是正负交替了，先负后正，先奇后偶，奇负偶正
    
    # 更新qw, *对应位置想乘，见RoPE公式
    q = q * cos_pos + q2 * sin_pos
    
    # k 同理
    k2 = torch.stack([-k[..., 1::2], k[..., 0::2]], dim=-1)
    k2 = k2.reshape(k.shape)
    k = k * cos_pos + k2 * sin_pos
    
    return q, k

### Attention

<img src="./images/attention.png" alt="示例图片" width="700">

$$
Attention(Q, K, V) = softmax(\frac{QK^\top}{\sqrt{d_k}})V
$$

In [45]:
def attention(q, k, v, mask=None, dropout=None, use_RoPE=True):
    # q.shape: (bs, head, seq_len, dk)
    # k.shape: (bs, head, seq_len, dk)
    # v.shape: (bs, head, seq_len, dk)
    
    if use_RoPE:
        q, k = RoPE(q, k)
    
    d_k = k.size()[-1]
    
    att_logits = torch.matmul(q, k.transpose(-2, -1)) # (bs, head, seq_len, seq_len)
    att_logits /= math.sqrt(d_k)
    
    if mask is not None:
        att_logits = att_logits.masked_fill(mask == 0, -1e9) # mask掉mask矩阵为0的部分，设为负无穷大
    
    att_scores = F.softmax(att_logits, dim=-1) # (bs, head, seq_ken, seq_len)
    
    if dropout is not None:
        att_scores = dropout(att_scores)
        
    # (bs, head, seq_ken, seq_len) * (bs, head, seq_ken, dk) = (bs, head, seq_ken, seq_len)
    
    return torch.matmul(att_scores, v), att_scores

In [46]:
if __name__ == '__main__':
    # (bs, head, seq_ken, dk)
    q = torch.randn((8, 12, 10, 32))
    k = torch.randn((8, 12, 10, 32))
    v = torch.randn((8, 12, 10, 32))
    
    res, att_scores = attention(q, k, v, mask=None, dropout=None, use_RoPE=True)
    
    # (bs, head, seq_len, seq_len), (bs, head, seq_len, seq_len)
    print(res.shape, att_scores.shape)

torch.Size([8, 12, 10, 32]) torch.Size([8, 12, 10, 10])


In [47]:
print(res)

tensor([[[[ 4.7924e-01, -8.1184e-02,  5.0471e-01,  ...,  3.3904e-02,
           -5.6309e-01, -6.9180e-01],
          [ 2.8582e-02, -6.0121e-01,  2.2196e-01,  ...,  5.8244e-01,
            4.9220e-02, -9.2528e-01],
          [-5.4878e-01,  1.3189e-01, -8.0629e-01,  ...,  5.6292e-01,
            4.6413e-01, -9.8989e-02],
          ...,
          [-9.2218e-02,  6.4448e-01, -8.5428e-02,  ..., -1.6412e-01,
            1.2261e+00,  2.9948e-02],
          [-3.7048e-01, -5.1464e-01, -3.4825e-02,  ...,  4.9220e-01,
            2.7366e-01, -6.0428e-01],
          [ 1.2036e-01, -3.5371e-01,  2.0824e-02,  ...,  8.7174e-01,
           -4.2703e-01, -1.4617e-01]],

         [[ 4.4240e-01,  2.7135e-01,  5.3776e-02,  ..., -4.9925e-01,
           -5.0704e-01, -2.3714e-01],
          [ 6.4617e-01, -1.4401e-03,  2.9460e-01,  ...,  4.6954e-01,
            2.9533e-02,  3.1978e-01],
          [ 1.3858e-01,  2.2991e-01, -3.6092e-02,  ...,  9.3772e-02,
           -4.0733e-01,  1.0657e-01],
          ...,
     

In [17]:
print(att_scores)

tensor([[[[0.0180, 0.0601, 0.0260,  ..., 0.3758, 0.1334, 0.0133],
          [0.3908, 0.0807, 0.0795,  ..., 0.0656, 0.0289, 0.0188],
          [0.3409, 0.0319, 0.1047,  ..., 0.0195, 0.0596, 0.1622],
          ...,
          [0.0573, 0.1623, 0.0544,  ..., 0.1582, 0.0978, 0.0624],
          [0.0164, 0.0631, 0.0293,  ..., 0.0773, 0.0669, 0.1870],
          [0.2110, 0.0840, 0.0599,  ..., 0.1923, 0.0307, 0.0273]],

         [[0.0153, 0.0116, 0.0195,  ..., 0.1708, 0.1476, 0.1923],
          [0.0374, 0.0141, 0.1279,  ..., 0.0389, 0.0975, 0.0265],
          [0.0181, 0.0448, 0.0415,  ..., 0.1504, 0.0865, 0.0359],
          ...,
          [0.0938, 0.0161, 0.0439,  ..., 0.1646, 0.2964, 0.0866],
          [0.0301, 0.0191, 0.0489,  ..., 0.1560, 0.4166, 0.1474],
          [0.0130, 0.0220, 0.0455,  ..., 0.1164, 0.0349, 0.0251]],

         [[0.1585, 0.1547, 0.0475,  ..., 0.0116, 0.1992, 0.1282],
          [0.0722, 0.1171, 0.1464,  ..., 0.1562, 0.1852, 0.0427],
          [0.3656, 0.1009, 0.0870,  ..., 0